In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup as bs
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
from pprint import pprint
import json 

## Start scraping process

### Getting all navigation of the website for access the elements

In [15]:
url = 'https://www.realestate.com.kh/buy/?page=1'
headers = {'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Mobile Safari/537.36'}

category_url = []

response = requests.get(url, headers=headers)
data = bs(response.content, 'html.parser')

# query all headers
get_all_header = data.find_all(class_='info listing')
get_all_header[0]

print(get_all_header[0])
# for i in get_all_header[0]:
#     print(i.find(class_='value').get_text())



<div class="info listing" href="/buy/kandaek/properties-dabest-153486/"><div class="copy-wrapper"><div class="heading">Villa<div class="price"><span class="value" style="text-decoration:">$130,000</span><span><div> </div><div class="percent listing"> </div></span></div></div><div class="address-features-block"><div class="address"><span class="location-pin"></span>Kandaek, Prasat Bakong, Siem Reap</div><div class="featured-box css-1ubmfrc eu11br40"><div class="private"><div class="feature"><span class="icon"></span> <span class="label">Bedrooms: 3</span><span class="short-label">3</span></div><div class="feature"><span class="icon"></span> <span class="label">Bathrooms: 3</span><span class="short-label">3</span></div><div class="feature"><span class="icon"></span> <span class="label">Land area: 240m²</span><span class="short-label">240m²</span></div></div></div></div><div class="listed"><svg class="css-ri58vd" viewbox="0 0 20 20"><path d="M10,0.4c-5.303,0-9.601,4.298-9.601,9.6c0,5.

In [33]:
print(get_all_header[0].find(class_='value').get_text())
print(get_all_header[0].find(class_='address').get_text().replace('\ue92f',''))
print(get_all_header[0].find(class_='heading').get_text().split('$')[0])
print(get_all_header[0].find_all(class_='label'))

$130,000
Kandaek, Prasat Bakong, Siem Reap
Villa
[<span class="label">Bedrooms: 3</span>, <span class="label">Bathrooms: 3</span>, <span class="label">Land area: 240m²</span>]


In [41]:
product_info = []

In [42]:
def get_product_info(url):
    browser = webdriver.Chrome(ChromeDriverManager().install())
    browser.get(url)
    headers = {'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Mobile Safari/537.36'}
    response = requests.get(url, headers=headers)
    data = bs(response.content, 'html.parser')

    # query all headers
    get_all_header_no_nest = data.find_all(class_='info listing')
    get_all_header_have_nest = data.find_all(class_='info project')
    time.sleep(5)
    for item in get_all_header_no_nest:
        product_info.append({'location' : item.find(class_='address').get_text().replace('\ue92f',''), 'type' : item.find(class_='heading').get_text().split('$')[0], 'price' : item.find(class_='value').get_text(), 'infos' : item.find_all(class_='label')})
    
    for item in get_all_header_have_nest:
        product_info.append({'location' : item.find(class_='address').get_text().replace('\ue92f',''), 'type' : item.find(class_='heading').get_text().split('$')[0], 'price' : item.find(class_='value').get_text(), 'infos' : item.find_all(class_='label')})

In [43]:
get_product_info(url)

C:\Users\Rax\AppData\Local\Temp\ipykernel_2460\3113621944.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


In [44]:
(product_info)

[{'location': 'Kandaek, Prasat Bakong, Siem Reap',
  'type': 'Villa',
  'price': '$130,000',
  'infos': [<span class="label">Bedrooms: 3</span>,
   <span class="label">Bathrooms: 3</span>,
   <span class="label">Land area: 240m²</span>]},
 {'location': 'Chreav, Siem Reap, Siem Reap',
  'type': 'Land',
  'price': '$59,150',
  'infos': [<span class="label">Land area: 455m²</span>]},
 {'location': 'Veal Vong, 7 Makara, Phnom Penh',
  'type': 'Condo',
  'price': '$220,000',
  'infos': [<span class="label">Bedrooms: 2</span>,
   <span class="label">Bathrooms: 2</span>,
   <span class="label">Car spaces: 1</span>,
   <span class="label">Floor area: 122m²</span>,
   <span class="label">Facing: E/S</span>]},
 {'location': 'DABEST PROPERTIES, Sala Kamraeuk, Siem Reap, Siem Reap',
  'type': 'Villa',
  'price': '$550,000',
  'infos': [<span class="label">Bedrooms: 6</span>,
   <span class="label">Bathrooms: 6</span>,
   <span class="label">Car spaces: 3</span>,
   <span class="label">Floor area: 